In [1]:
import pandas as pd

import gc
####

import seaborn as sns
import plotly.express as px

import numpy as np

import sqlite3
from sqlite3 import Error
import os
import re, string
from nltk.corpus import stopwords
import nltk
import matplotlib.pyplot as plt

import torch
#from transformers import AutoTokenizer, BertForSequenceClassification
## felipe


from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA

nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


import pandas as pd
import re, string


pd.options.display.max_columns = None

#from sentence_transformers import SentenceTransformer, util
#from transformers import *

import spacy

nlp = spacy.load('es_core_news_sm', disable=['tagger', 'parser', 'ner'])



from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import preprocessing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [20]:
!python -m spacy download es_core_news_lg

2023-07-11 15:33:18.980118: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 2.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')


In [ ]:
!python -m spacy download es_core_news_lg

%pip install pycorrector
%pip install unidecode
%pip install pyspellchecker

# etl

In [2]:
import pandas as pd
import sqlite3

In [3]:


query1 = "SELECT NUMERO_PQR,DESCRIPCION_PQR FROM pqr_priorizar where MEDIO_RECEPCION !=  'VIRTUAL' and unico=1 and SINTOMA_PQR ='CONCEPTOS FACTURADOS' and FECHA_CREACION > '2023-07-03' and TECNOLOGIA ='FO'"

query2 = "(SELECT * FROM altas where ST_TIPO_SERVICIO='FTTH' AND FASE='ORDERF' AND (tipoTramite= 'VTA' OR tipoTramite=  'Sales')) AS myquery"

In [4]:

# Connect to the SQLite database file
conn = sqlite3.connect('/content/on2/pqr_priorizar.db')

# Read the SQL query into a pandas DataFrame

pqrs = pd.read_sql_query(query1, conn)

# Close the database connection
conn.close()

# Print the DataFrame
pqrs

#MDM-PQR-39191683

,NUMERO_PQR,DESCRIPCION_PQR
0,MDM-PQR-39701420,CC: 1015411092 NOMBRE: JULIAN FERNANDO RICO ...
1,MDM-PQR-39704380,Nombre: INGRID JOHANNA VASQUEZ GONZALEZ Ced...
2,MDM-PQR-39704437,Nombre: DIANA PATRICIA MERA GARZON Cedula: ...
3,MDM-PQR-39704443,RRSS//TWSweetNata2 //@ETBsoluciones su servici...
4,MDM-PQR-39704595,Cliente con dudas sobre el valor que le arroja...
...,...,...
2763,MDM-PQR-39818416,Línea: 6012494604 Nombre de quien se comunica...
2764,MDM-PQR-39818426,NOMBRE : JUAN VIANNEY PEREZ TICORA CC: 14267...
2765,MDM-PQR-39818642,ínea: 6016738015 Nombre: RUTH JANNETH MORENO T...
2766,MDM-PQR-39818688,se valida cliente con pago anulado del mes de...


# TRASNFORMAR LAS DATOS DE TEXTO

In [6]:
sys.path.append('../librerias')

In [7]:
from clean_text import clean_text

entro2


In [8]:
pqrs = pqrs.fillna("vacio")



In [9]:
pqrs.isnull().sum()

NUMERO_PQR         0
DESCRIPCION_PQR    0
dtype: int64

In [10]:
#declare answers and questions
descripcion=pqrs.DESCRIPCION_PQR





#clean_descripcion=[k for k in clean_descripcion if 'soporte'  in k]
clean_descripcion = [clean_text(des) for des in descripcion]


len(clean_descripcion)

2768

In [11]:
clean_descripcion[0]

'  quiere verificar las facturas '

spelling

In [12]:
from tqdm import tqdm
from spelling import correct_sentence


hola


In [13]:
corrected_sentences = []
for text in tqdm(clean_descripcion, desc="Processing"):
    corrected_sentence = correct_sentence(text)
    corrected_sentences.append(corrected_sentence)

clean_descripcion=corrected_sentences

Processing: 100%|██████████| 2768/2768 [08:41<00:00,  5.31it/s] 


lemma token stemm

In [14]:
from lemastem import data_preprocessing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


['yanguma', 'yessenia', 'tirano', 'thania', 'liliam', '’', 'hita', 'anni', 'cajiao', 'melody', 'ofelma', 'feet', 'erlinda', 'correales', 'yovani', 'amin', 'magnus', 'heyda', 'aza', 'erick', 'eloisa', 'liborio', 'krystel', 'lorenzo', 'heiler', 'zorrilla', 'hubieran', 'brandon', 'monta?o', 'sarha', 'livingston', 'fueron', 'hermofilo', 'atilia', 'zona', 'arcangel', 'mari', 'cajica', 'lelis', 'emiliano', 'jeimmy', 'yosmary', 'yomayusa', 'joana', 'johanna', 'kateryn', 'lugo', 'medica', 'pernett', 'giuliana', 'emiliana', 'tacuma', 'grosso', 'número', 'divia', 'luzdary', 'gerson', 'jonh', 'adarme', 'giraldo', 'mariann', 'froilan', 'mayibe', 'neiva', 'epaminondas', 'lirio', 'rolfe', 'turizo', 'zoila', 'santander', 'oswaldo', 'bertilda', 'borbon', 'tropical', 'sammy', 'alaguna', 'katharina', 'parraga', 'sabbagh', 'miguez', 'esos', 'eventos', 'importaciones', 'viveros', 'maxima', 'lesli', 'arbey', 'angelly', 'lavaseco', 'mercados', 'wilfer', 'wilmar', 'quinayas', 'jennyfer', 'siza', 'ermenson', 

In [15]:

clean_descripcion=data_preprocessing(clean_descripcion)
clean_descripcion[190]

['solicitar',
 'aclaracion',
 'factura',
 'informar',
 'genero',
 'cobro',
 'reconeporion',
 'descuento',
 'ultimo',
 'facturacion']

In [16]:
clean_descripcion[0]

['querer', 'verificar', 'factura']

In [17]:
merged_data=[]
for i in range(len(clean_descripcion)):
    merged_data.append(" ".join(clean_descripcion[i]))

merged_df=pd.DataFrame(merged_data,columns =['descripcion'])


In [18]:
from unidecode import unidecode

def remove_accents(text):
    return unidecode(text)

# Aplicar la función a la columna 'Texto'
merged_df['descripcion'] = merged_df['descripcion'].apply(remove_accents)

In [19]:
merged_df

,descripcion
0,querer verificar factura
1,validar facturacion contar saldo favor
2,validar facturacion solicito tramite cancelaci...
3,rrsstwsweetnata etbsolucion no increible compl...
4,duda arrojar contar cancelado
...,...
2763,concepto facturar
2764,incremento factura
2765,solicitar saber llegar factura tanto
2766,validar pago anulado cancelar llegar factura c...



clean_descripcion = [text for text in clean_descripcion if "svas" not in text and "directv" not in text  and "winsport" not in text and "hbo" not in text  ]

len(clean_descripcion)

In [20]:
from tfidf import TFIDF
import joblib

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from joblib import dump


X = merged_df['descripcion']
X_test=merged_df['descripcion']


X_train,_ = TFIDF(X,X_test)

# Create a new instance of PCA with the optimal number of components

pca = joblib.load('/content/PRIORIZAR/modelo/pca_binario.joblib')

#pca = PCA(n_components=585)


X_train_new = pca.fit_transform(X_train)


print("train with old features: ",np.array(X).shape)
print("train with new features:" ,np.array(X_train_new).shape)



tf-idf with 1301 features
train with old features:  (2768,)
train with new features: (2768, 804)


# cargar modelo

In [22]:
from joblib import dump

import joblib


model = joblib.load('/content/PRIORIZAR/modelo/modelo_binario.joblib')

print(model)

LGBMClassifier(bagging_fraction=0.9747337180903012, bagging_freq=4,
               feature_fraction=0.6526645750030313,
               lambda_l1=0.02873208934418634, lambda_l2=2.3446621696223424,
               learning_rate=0.0815126268370284, min_child_samples=35,
               n_estimators=88, num_leaves=35)


In [23]:
y_pred = model.predict(X_train_new)

pqrs['prediccion_algoritmo'] = y_pred

pqrs['descripcion_clean']=merged_df['descripcion']



# Lista de palabras a filtrar
words_to_remove = ['svas', 'directv', 'paramont', 'hbo']

# Filtrar las filas del DataFrame
c_facturados_correcion = pqrs[~pqrs['descripcion_clean'].str.contains('|'.join(words_to_remove))]

c_facturados_SVAS = pqrs[pqrs['descripcion_clean'].str.contains('|'.join(words_to_remove))]



c_facturados_correcion


,NUMERO_PQR,DESCRIPCION_PQR,prediccion_algoritmo,descripcion_clean
0,MDM-PQR-39701420,CC: 1015411092 NOMBRE: JULIAN FERNANDO RICO ...,1.0,querer verificar factura
1,MDM-PQR-39704380,Nombre: INGRID JOHANNA VASQUEZ GONZALEZ Ced...,1.0,validar facturacion contar saldo favor
3,MDM-PQR-39704443,RRSS//TWSweetNata2 //@ETBsoluciones su servici...,1.0,rrsstwsweetnata etbsolucion no increible compl...
4,MDM-PQR-39704595,Cliente con dudas sobre el valor que le arroja...,1.0,duda arrojar contar cancelado
5,MDM-PQR-39704674,NOMBRE.jairo C.C. 1078578549 TEL.3132226145 AS...,1.0,c c asunto saber factura llego mas costoso cua...
...,...,...,...,...
2763,MDM-PQR-39818416,Línea: 6012494604 Nombre de quien se comunica...,1.0,concepto facturar
2764,MDM-PQR-39818426,NOMBRE : JUAN VIANNEY PEREZ TICORA CC: 14267...,1.0,incremento factura
2765,MDM-PQR-39818642,ínea: 6016738015 Nombre: RUTH JANNETH MORENO T...,1.0,solicitar saber llegar factura tanto
2766,MDM-PQR-39818688,se valida cliente con pago anulado del mes de...,1.0,validar pago anulado cancelar llegar factura c...


In [24]:
from datetime import date

today = date.today()

year, week_num, day_of_week = today.isocalendar()





In [43]:
errores = len(c_facturados_correcion[c_facturados_correcion['prediccion_algoritmo'] == 0])
total  =len(c_facturados_correcion)
sva=len(c_facturados_SVAS)
error=(errores/total)*100
semana=week_num-1
index='-'.join(map(str, [year,semana]))



data = {'errores': [errores],
        'total': [total],
        'sva': [sva],
        '%_error':[error],
        'semana':[semana],
        'index':[index]}

tracking_tipificacion = pd.DataFrame(data)


#tracking_c_facturado

In [44]:
tracking_tipificacion

,errores,total,sva,%_error,semana,index
0,578,2686,82,21.518987,27,2023-27


cargando data apra tablero

In [45]:

 
# append data frame to CSV file
tracking_tipificacion.to_csv('/content/tracking_c_facturado.csv', mode='a', index=False, header=False)
 
# print message

!rclone copy  /content/tracking_c_facturado.csv one:/analitica_datos/tracking_c_facturado

In [39]:
import pandas as pd
from datetime import date


from datetime import datetime

# Obtener la fecha y hora actual
current_date = datetime.now()
# Get the current date
# Specify the root directory path
root_path = '/content/PRIORIZAR/SALIDAS/'

# Export the DataFrame to a CSV file with the root path and current date in the filename
filename1 = f"{root_path}c_facturados_correcion_{current_date}.csv"
filename2 = f"{root_path}c_facturados_SVAS_{current_date}.csv"


c_facturados_correcion.to_csv(filename1, index=False)

c_facturados_SVAS.to_csv(filename2, index=False)




#pandas_c_F.to_csv('c_facturados.csv', index=False)

print("DONE")

DONE


# modelo retipificacion

filter data with 0 label

In [46]:
retipificacion = c_facturados_correcion[c_facturados_correcion['prediccion_algoritmo'] == 0.0]
retipificacion

,NUMERO_PQR,DESCRIPCION_PQR,prediccion_algoritmo,descripcion_clean
9,MDM-PQR-39704947,Nombre Luis Cedula 79604317 Celular 3107697627...,0.0,suspendido pago no entender si pago factura co...
19,MDM-PQR-39705614,Linea: Nombre: Nubia castellanos (esposa ) ...,0.0,informar llego factura
26,MDM-PQR-39705880,Linea: Nombre: ANA ROSA VIVAS HERNANDEZ ...,0.0,fibra operar gratis
27,MDM-PQR-39705953,linea: nombre de quien se comunica: Mauricio c...,0.0,solicitar referente factura
33,MDM-PQR-39706264,Linea: 6017906523 Nombre de quien se comunica:...,0.0,solicitar aclaracion factura
...,...,...,...,...
2755,MDM-PQR-39817311,Nombre:ANA SOFIA PARRA CC:20163746 Celula...,0.0,momento factura generar cobro
2756,MDM-PQR-39817526,Se comunica GUILLERMO ANDRES HERNANDEZ ZORRO; ...,0.0,wifi no conectar no navegar realizar guiado pl...
2757,MDM-PQR-39817557,NombreYEIMI ANDREA PEDRAZA VARGAS Celular 320...,0.0,solicitar respecto factura
2762,MDM-PQR-39818394,NOMBRE: ALEJANDRO ALVAREZ RIASCOS CEDULA: 10...,0.0,hacer llego factura no


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split



X = retipificacion['descripcion_clean']
X_test=retipificacion['descripcion_clean']


X_train,_ = TFIDF(X,X_test)

# Create a new instance of PCA with the optimal number of components

pca = joblib.load('/content/PRIORIZAR/modelo/pca_retipificacion.joblib')

#pca = PCA(n_components=361)


X_train_new = pca.fit_transform(X_train)


print("train with old features: ",np.array(X).shape)
print("train with new features:" ,np.array(X_train_new).shape)



tf-idf with 836 features
train with old features:  (1180,)
train with new features: (1180, 299)


In [28]:
model = joblib.load('/content/PRIORIZAR/modelo/retipificacion.joblib')

print(model)

LGBMClassifier(bagging_fraction=0.9030872285178855, bagging_freq=3,
               feature_fraction=0.8256242530913764,
               lambda_l1=0.049148437664035254, lambda_l2=1.8266591621558763,
               learning_rate=0.09934154462012913, min_child_samples=23,
               n_estimators=41, num_leaves=33)


In [29]:
y_pred = model.predict(X_train_new)

retipificacion["retipificacion_algoritmo"]=y_pred

<ipython-input-29-9505f33dc576>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retipificacion["retipificacion_algoritmo"]=y_pred


mapping = {'otros': 0, 'INCREMENTO DE TARIFA': 1, 'ASEGURAMIENTO': 2,  'BENEFICIO DE RETENCION': 3,'compenscion': 4}


In [30]:
retipificacion

,NUMERO_PQR,DESCRIPCION_PQR,prediccion_algoritmo,descripcion_clean,retipificacion_algoritmo
0,MDM-PQR-39565361,CLIENTE SE COMUNICA POR INFORMACIÓN SOBRE SU F...,0,unicar factura concepto aclarar,4
4,MDM-PQR-39565657,NURY EDELMIRA MORENO MORENO //51563810 //60136...,0,poder generar ajuste decir parecer pagar llama...,1
6,MDM-PQR-39566135,Línea: 6013644591 Nombre: ANA BETRIZ OSORIO CU...,0,solicitar saber llegar factura tanto si decir ...,1
7,MDM-PQR-39566173,Nombre: MARTIN FLOREZ MORALES CC: 80368377 C...,0,facturacion,2
9,MDM-PQR-39566352,Nombre: JOSE DELFIN TOVAR CC: 19375096 Celular...,0,,3
...,...,...,...,...,...
3055,MDM-PQR-39736638,CLIENTE SOLICITA ACLARAR LOS CONCEPTOS FACTURA...,0,,3
3057,MDM-PQR-39736819,Línea: 6014104197 Nombre del Cliente: GUILLE...,0,parecer querer saber si aparecer,1
3059,MDM-PQR-39737146,envio de factura correcta ya que en extemprane...,0,parecer,1
3061,MDM-PQR-39737284,NombreMARIA NELLY RODRIGUEZ ANDRADE Celular 3...,0,envio facturacion ajustado,2


In [31]:
import pandas as pd
from datetime import date


from datetime import datetime

# Obtener la fecha y hora actual
current_date = datetime.now()
# Get the current date
# Specify the root directory path
root_path = '/content/PRIORIZAR/SALIDAS/'

# Export the DataFrame to a CSV file with the root path and current date in the filename
filename3 = f"{root_path}c_facturados_retipificacion_{current_date}.csv"



retipificacion.to_csv(filename3, index=False)





#pandas_c_F.to_csv('c_facturados.csv', index=False)

print("DONE")

DONE
